In [1]:
## Uge 11

## notebook_5 er at finde sansynlighed for occlusion med bayes

import pandas as pd
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv("./data/train/train.csv")
test_data = pd.read_csv("./data/test/test.csv")
train_data.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [3]:
df = train_data.copy()
X = df[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Info', 'Blur', 'Pawpularity']]
y = df['Occlusion']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [6]:
from sklearn.metrics import classification_report


y_prob = gnb.predict_proba(X_test)

# Extract the probability of 'Occlusion'
occlusion_prob = y_prob[:, 1]  # Assuming 'Occlusion' is the second class

print(occlusion_prob)

[0.89947576 0.99865481 0.08373513 ... 0.77148326 0.89602426 0.86074353]


In [7]:
# Create a DataFrame from the probabilities
prob_df = pd.DataFrame(occlusion_prob, columns=['Occlusion_Prob'])

# Reset index of your test data
X_test_reset = X_test.reset_index(drop=True)

# Concatenate the probabilities with the original data
result_df = pd.concat([X_test_reset, prob_df], axis=1)

print(result_df)

      Subject Focus  Eyes  Face  Near  Action  Accessory  Group  Collage  \
0                 0     1     1     1       0          0      0        0   
1                 0     1     1     0       0          0      0        1   
2                 0     1     1     1       0          1      0        0   
3                 0     0     1     1       0          0      1        0   
4                 0     1     1     1       0          0      1        0   
...             ...   ...   ...   ...     ...        ...    ...      ...   
1978              0     1     1     1       0          0      0        0   
1979              0     1     1     1       0          0      0        0   
1980              0     0     0     1       0          0      0        0   
1981              0     1     1     1       0          0      0        0   
1982              0     1     1     1       0          0      0        0   

      Human  Info  Blur  Pawpularity  Occlusion_Prob  
0         0     0     0         

In [80]:
# Filter rows where n features = 1 or 0
# Givet valgte features i vores applikation, så kan vi udregne sandsynligheden for at en occlusion er tilstede(dog bare en test))
result_df_human = result_df[(result_df['Near'] == 1) & (result_df['Human'] == 1)]

print(result_df_human.head())

     Subject Focus  Eyes  Face  Near  Action  Accessory  Group  Collage  \
6                0     1     1     1       1          0      0        0   
527              0     1     1     1       1          1      0        0   
889              0     0     1     1       1          0      0        0   
932              0     1     1     1       1          0      0        0   
933              1     1     1     1       1          0      0        0   

     Human  Info  Blur  Pawpularity  Occlusion_Prob  
6        0     0     0           36    1.183315e-05  
527      0     0     0            2    1.030938e-07  
889      0     0     0           52    9.232862e-06  
932      1     0     0           65    5.960541e-02  
933      0     0     0           17    0.000000e+00  
